In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/teknofest_nlp/teknofest.csv', delimiter='|')

#TWEET ÖN İŞLEME
import re

def clean_tweet(tweet):
    # Remove non-alphanumeric characters (including numbers) and convert to lowercase
    tweet = re.sub(r'[^a-zA-ZğüşıöçĞÜŞİÖÇ\s]', '', tweet.lower())
    
    # Tokenize tweet
    tokens = tweet.split()
    
    # Take first 5 letters of each word
    tokens = [word[:5] for word in tokens]
    
    # Join tokens back into a string
    clean_tweet = ' '.join(tokens)
    
    return clean_tweet

df['text'] = df['text'].apply(lambda x: clean_tweet(x))

#DATAYI BALANCE HALE GETİRMEK İÇİN TRAİN_DF ÜZERİNDEN SINIFLARA BÖLME
train_df = df.copy()
from sklearn.utils import resample
minority_class = train_df[train_df.is_offensive == 0]#OFENSİF OLMAYANLAR
insult_class = train_df[train_df.target == 'INSULT']#DİĞEr DÖRT SINIF
racist_class = train_df[train_df.target == 'RACIST']
sexist_class = train_df[train_df.target == 'SEXIST']
profanity_class = train_df[train_df.target == 'PROFANITY']

#HER SINIFTAN (NEGATİF TWEETLER İÇİN) 900 VERİ ALMA + POZİTİF TWEETLER

n_samples = 900
insult_class_resampled = resample(insult_class, replace=False, n_samples=n_samples, random_state=42)
racist_class_resampled = resample(racist_class, replace=False, n_samples=n_samples, random_state=42)
sexist_class_resampled = resample(sexist_class, replace=False, n_samples=n_samples, random_state=42)
profanity_class_resampled = resample(profanity_class, replace=False, n_samples=n_samples, random_state=42)

#NAİVE BAYES MODELİ İÇİN BALANCE DATAFRAME OLUŞTURMA
offensive_df = pd.concat([minority_class, insult_class_resampled,racist_class_resampled,sexist_class_resampled,profanity_class_resampled])

#BERT İÇİN SADECE NEGATİF VERİLER İLE DATAFRAME OLUŞTURMA
negative_df = train_df[train_df['target'] != 'OTHER']

#VERİLERİ SHUFFLE İLE KARIŞTIRIYORUZ
from sklearn.utils import shuffle

offensive_df = shuffle(offensive_df, random_state=42)
negative_df = shuffle(negative_df, random_state=42)

In [ ]:
#Naive Bayes MODELİ TRAİN KISMI

X_train = offensive_df['text']
y_train = offensive_df['is_offensive']

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)

from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train, y_train)

#NB MODELİNİ KAYDETME
import pickle
with open('nb_model.pkl', 'wb') as f:
    pickle.dump(clf, f)


In [ ]:
#NEGATİF TWEETLERİ BERT MODELİNE UYGUN HALE GETİRME
import pandas as pd

target_mapping = {"INSULT": 1, "RACIST": 2, "SEXIST": 3, "PROFANITY": 4}
negative_df["target"] = negative_df["target"].map(target_mapping)


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 25.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelForSequenceClassification

tokenizer= AutoTokenizer.from_pretrained("savasy/bert-turkish-text-classification")
model= AutoModelForSequenceClassification.from_pretrained("savasy/bert-turkish-text-classification")

import numpy as np
from transformers import AutoTokenizer

train_encodings = tokenizer(list(negative_df['text']), truncation=True, padding=True)
train_labels = np.array(list(negative_df['target']))

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.train()

train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels))

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optim.step()

#BERT MODELİNİ KAYDETME

with open('bert_model.pkl', 'wb') as f:
    pickle.dump(model, f)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 

In [ ]:

#ÖNİŞLEME FONKSİYONU TEST VERİSİ İÇİN
def preprocess_tweet(tweet):
    # Remove non-alphanumeric characters (including numbers) and convert to lowercase
    tweet = re.sub(r'[^a-zA-ZğüşıöçĞÜŞİÖÇ\s]', '', tweet.lower())
    # Tokenize tweet
    tokens = tweet.split()
    # Take first 5 letters of each word
    tokens = [word[:5] for word in tokens]
    # Join tokens back into a string
    clean_tweet = ' '.join(tokens)
    return ' '.join(tokens)

In [ ]:
pip install gradio


In [ ]:
import pandas as pd
from pandas import DataFrame
import gradio as gr
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import gensim
from gensim.models import Word2Vec
import pandas as pd
from sklearn.utils import resample
from sklearn.naive_bayes import MultinomialNB
import re
from sklearn.feature_extraction.text import CountVectorizer

def auth(username, password):
    if username == "MSKU-CENG-NLP-2" and password == "JMTU7FVNKFKFKW4J":
        return True
    else:
        return False

#ÖNİŞLEME FONKSİYONU TEST VERİSİ İÇİN
def preprocess_tweet(tweet):
    # Remove non-alphanumeric characters (including numbers) and convert to lowercase
    tweet = re.sub(r'[^a-zA-ZğüşıöçĞÜŞİÖÇ\s]', '', tweet.lower())
    # Tokenize tweet
    tokens = tweet.split()
    # Take first 5 letters of each word
    tokens = [word[:5] for word in tokens]
    # Join tokens back into a string
    clean_tweet = ' '.join(tokens)
    return ' '.join(tokens)

def predict(df):
    output_file = "output_MSKU-CENG-NLP-2.csv"

    df = pd.read_csv(df.name, sep="|")
    #ÖNİŞLEMENİN TEST VERİSETİNE UYGULANMASI
    df=df.drop(['offansive'],axis=1)
    df=df.drop(['target'],axis=1)
    df['text'] = df['text'].apply(preprocess_tweet)
    
    #KAYDEDİLEN MODELLERİN YÜKLENMESİ
    with open('nb_model.pkl', 'rb') as f:
      nb_loaded_model = pickle.load(f)

    with open('bert_model.pkl', 'rb') as f:
      bert_loaded_model = pickle.load(f)

    #NB PREDİCTİON
    X_test = df['text']


    X_test = vectorizer.transform(X_test)

    nb_pred = nb_loaded_model.predict(X_test)
    df['offansive'] = nb_pred
       
    #BERTE SOKULACAK NEGATİF ETİKETLİ TWEETLER İÇİN YENİ DATAFRAME OLUŞTURMA
    neg_results = df[df['offansive'] == 1]
    
    #BERT TEST
    bert_loaded_model.eval()
    test_encodings = tokenizer(list(neg_results['text']), truncation=True, padding=True)
    test_dataset = torch.utils.data.TensorDataset(
        torch.tensor(test_encodings['input_ids']),
        torch.tensor(test_encodings['attention_mask']))

    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=16,
        shuffle=False)

    # TEST DATA İLE PREDİCTİON
    predictions = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask = tuple(t.to(device) for t in batch)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions.extend(torch.argmax(logits, axis=1).tolist())

    neg_results['target'] = predictions
  
    #VERİLEN DF İLE NEGATİF ETİKETLİ TWEETLERİN 4 SINIF TAHMİNLERİNİ BİRLEŞTİRME
    df = pd.merge(df, neg_results[['id', 'target']], on='id', how='left')

    #TARGET COLUMNDAKİ BOŞLUKLARI OTHER İLE DOLDURMA
    df['target'] = df['target'].fillna('OTHER')

    #UYGUN ÇIKTI İÇİN TARGET MAP UYGULAMASI
    target_mapping = {1: "INSULT", 2: "RACIST", 3: "SEXIST", 4: "PROFANITY", 'OTHER' : 'OTHER'}
    df["target"] = df["target"].map(target_mapping)

    #TAHMİNLERİ CSV HALİNE GETİRME
    df.to_csv(output_file, index=False, sep="|")
    return (output_file)

iface = gr.Interface(predict, "file", "file")

if __name__ == "__main__":
    iface.launch(share=True, auth=auth,debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://78ca3fe5f88ca289cc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


<ipython-input-10-f1c7ea95d6b5>:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_results['target'] = predictions
<ipython-input-10-f1c7ea95d6b5>:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_results['target'] = predictions
<ipython-input-10-f1c7ea95d6b5>:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui